# Fresh Products

In [132]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns
import sys

# Move two levels up (to the project root) and append the `src` folder
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))

# Append src to sys.path
sys.path.append(src_path)

from src import query_engines, dataframe_visualizer

q = query_engines.QueryEngines()

## Q1: Identify all kinds of segmentation

In this first part I want to segment all the possible partners. In Specialties, TP, NTP, MFCs and All groceries. We also check what is the perc of migrated Store Address Ids for each of them.

### What is the distribution of segments_lvl 1?

In [48]:
q.prepare_query(
    'possible_segmentations.sql'
    ,params=None
    ,to_load_file='possible_segmentations'
    ,load_from_to_load_file='possible_segmentations'
)

possible_segmentations  = q.query_run_starburst()

grouped = possible_segmentations.groupby('segment_1').agg(
    distinct_sad_id = ('store_address_id','nunique'),
    migrated_sad_id = ('is_migrated', lambda x: x.sum())
).reset_index()

grouped['perc_m'] = grouped['migrated_sad_id']/grouped['distinct_sad_id']
grouped.sort_values(by='distinct_sad_id', ascending=False)

,segment_1,distinct_sad_id,migrated_sad_id,perc_m
2,Specialties,15387,1442,0.093715
3,Top Partner,10546,8496,0.805614
1,Non Top Partner,5850,1682,0.287521
0,MFC,256,245,0.957031


### What is the distribution of segment_lvl 2?

In [49]:
grouped = possible_segmentations.groupby('segment_2').agg(
    distinct_sad_id = ('store_address_id','nunique'),
    migrated_sad_id = ('is_migrated', lambda x: x.sum())
).reset_index()

grouped['perc_m'] = grouped['migrated_sad_id']/grouped['distinct_sad_id']
grouped.sort_values(by='distinct_sad_id', ascending=False)

,segment_2,distinct_sad_id,migrated_sad_id,perc_m
0,Groceries Partner,16396,10178,0.620761
2,Specialties,15387,1442,0.093715
1,MFC,256,245,0.957031


# Q2: % of users that have placed a FO

In [146]:
START_DATE = "'2024-07-01'"
END_DATE = "'2024-09-15'"

params = [
    {'name':'start_date','value':START_DATE},
    {'name':'end_date','value':END_DATE}
]

q.prepare_query(
    'customer_categories.sql'
    ,params=params
    ,to_load_file='customer_categories'
    ,load_from_to_load_file='customer_categories'
)

ccat  = q.query_run_starburst()
ccat

,country,segment_1,segment_2,all_customers,f_customers,nf_customers,all_orders,f_orders,nf_orders,all_gmv,...,f_feedback_orders_retained_Store,nf_feedback_orders_retained_Store,all_feedback_orders_fretained_Groceries,f_feedback_orders_fretained_Groceries,nf_feedback_orders_fretained_Groceries,all_feedback_orders_fretained_Store,f_feedback_orders_fretained_Store,nf_feedback_orders_fretained_Store,check1,check2
0,MD,Top Partner,Groceries Partner,5272,4071,1999,13565,9880,3685,2.602527e+05,...,28,1,25,25,0,22,22,0,110985,110985
1,CI,Specialties,Specialties,2360,1149,1382,4259,2054,2205,8.024415e+04,...,3,0,5,5,0,2,2,0,10611,10611
2,BA,Top Partner,Groceries Partner,3891,2817,1674,9688,6804,2884,1.965560e+05,...,13,0,18,18,0,12,12,0,86633,86633
3,MA,Top Partner,Groceries Partner,126469,97766,69591,536347,371308,165039,8.390579e+06,...,1669,303,1999,1999,0,1356,1356,0,3753134,3753134
4,KG,Top Partner,Groceries Partner,9132,5420,5224,21075,11699,9376,1.975718e+05,...,46,10,51,51,0,35,35,0,105999,105999
5,RO,Top Partner,Groceries Partner,182895,139451,91373,818139,596795,221344,2.292271e+07,...,1478,78,1852,1852,0,1285,1285,0,7384346,7384346
6,UA,Specialties,Specialties,1310,413,1039,2951,700,2251,6.616051e+04,...,2,0,4,4,0,2,2,0,8051,8051
7,KZ,MFC,MFC,15905,9966,10696,58116,28316,29800,6.417096e+05,...,98,18,92,92,0,71,71,0,253167,253167
8,KG,MFC,MFC,9280,4719,6532,27443,12572,14871,3.424424e+05,...,63,9,50,50,0,45,45,0,123448,123448
9,MA,MFC,MFC,75978,42704,54033,246536,124353,122183,3.114198e+06,...,1064,375,1106,1106,0,844,844,0,1237479,1237479


In [145]:
ccat.sum()

country                                    MDCIBAMAKGROUAKZKGMAHRPLMEKZTNPTITBGPLGEUAITME...
segment_1                                  Top PartnerSpecialtiesTop PartnerTop PartnerTo...
segment_2                                  Groceries PartnerSpecialtiesGroceries PartnerG...
all_customers                                                                        1761768
f_customers                                                                          1249169
nf_customers                                                                          890425
all_orders                                                                           5902652
f_orders                                                                             3976464
nf_orders                                                                            1926188
all_gmv                                                                      129696918.97679
fo_gmv                                                                